In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv3D, MaxPool3D, Conv2D, MaxPool2D
from tensorflow.keras.layers import Flatten

In [2]:
from tensorflow.keras.backend import manual_variable_initialization 
manual_variable_initialization(True)

In [1]:
def model_builder(n_filter,n_hidden,neuron,act,kernel):
    model = Sequential()

    n_conv = 2
    for j in range(n_conv):
        model.add(Conv2D(filters=n_filter,kernel_size=(32,kernel),activation='relu',padding='valid'))
        model.add(Conv2D(filters=n_filter,kernel_size=(32,kernel),activation='relu',padding='valid'))
    model.add(Flatten())
    for i in range(n_hidden):
        model.add(Dense(neuron,activation=act))

    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss='binary_crossentropy', metrics= ['accuracy'])    
    return model

In [34]:
#Load the trained ML model
model1 = model_builder(32,4,400,'relu',3)
model1.load_weights('2302/')

In [23]:
import ifxdaq
import processing
import numpy as np
#print(ifxdaq.__version__)
from ifxdaq.sensor.radar_ifx import RadarIfxAvian

config_file = "radar_configs/RadarIfxBGT60.json"
raw_data    = []

In [26]:
raw_data = []
time = 10000
with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
    
    for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
        
        raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
        
        if(len(raw_data) > time-1 and len(raw_data) % time == 0):        # 5000 is the number of frames. which corresponds to 5seconds
            
            data = np.swapaxes(np.asarray(raw_data), 0, 1)
            
            range_data, range_idx = processing.processing_rangeData(data)
            
            
            raw = np.abs(np.expand_dims(range_data,axis=3)) * 20
            
            pred = model1.predict(raw[:,0:-1:10,:,:])
            
            if sum((pred >= 0.5)*1) > 1:
                moving = True
                print('Subject is in movement')
            else:
                moving = False
                print('Subject is resting')
        
        raw_data = []